### Imports


In [307]:
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score

# PyTorch
torch.manual_seed(42)
torch.cuda.manual_seed_all(42)

# NumPy
np.random.seed(42)

tf.random.set_seed(42)


## Data Preprocessing

In [308]:
df = pd.read_csv('Churn_Modelling.csv')


In [309]:
df.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [310]:
df.describe()

,RowNumber,CustomerId,CreditScore,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
count,10000.00000,1.000000e+04,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.00000,10000.000000,10000.000000,10000.000000
mean,5000.50000,1.569094e+07,650.528800,38.921800,5.012800,76485.889288,1.530200,0.70550,0.515100,100090.239881,0.203700
std,2886.89568,7.193619e+04,96.653299,10.487806,2.892174,62397.405202,0.581654,0.45584,0.499797,57510.492818,0.402769
min,1.00000,1.556570e+07,350.000000,18.000000,0.000000,0.000000,1.000000,0.00000,0.000000,11.580000,0.000000
25%,2500.75000,1.562853e+07,584.000000,32.000000,3.000000,0.000000,1.000000,0.00000,0.000000,51002.110000,0.000000
50%,5000.50000,1.569074e+07,652.000000,37.000000,5.000000,97198.540000,1.000000,1.00000,1.000000,100193.915000,0.000000
75%,7500.25000,1.575323e+07,718.000000,44.000000,7.000000,127644.240000,2.000000,1.00000,1.000000,149388.247500,0.000000
max,10000.00000,1.581569e+07,850.000000,92.000000,10.000000,250898.090000,4.000000,1.00000,1.000000,199992.480000,1.000000


In [311]:
df.isnull().sum()

,0
RowNumber,0
CustomerId,0
Surname,0
CreditScore,0
Geography,0
Gender,0
Age,0
Tenure,0
Balance,0
NumOfProducts,0


In [312]:
# row number , customer id and surname dont play any rule in churn modelling
X = df.drop(['RowNumber','CustomerId','Surname','Exited'],axis=1)
y = df['Exited']


In [313]:
print(X.shape)
print(y.shape)
print(X)
print(y)

(10000, 10)
(10000,)
      CreditScore Geography  Gender  Age  Tenure    Balance  NumOfProducts  \
0             619    France  Female   42       2       0.00              1   
1             608     Spain  Female   41       1   83807.86              1   
2             502    France  Female   42       8  159660.80              3   
3             699    France  Female   39       1       0.00              2   
4             850     Spain  Female   43       2  125510.82              1   
...           ...       ...     ...  ...     ...        ...            ...   
9995          771    France    Male   39       5       0.00              2   
9996          516    France    Male   35      10   57369.61              1   
9997          709    France  Female   36       7       0.00              1   
9998          772   Germany    Male   42       3   75075.31              2   
9999          792    France  Female   28       4  130142.79              1   

      HasCrCard  IsActiveMember  Estimated

In [314]:
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder

ct = ColumnTransformer(transformers=[('encoder', OneHotEncoder(), [1])], remainder='passthrough')
## encoding geo which is column 1

In [315]:
X

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,Female,42,2,0.00,1,1,1,101348.88
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58
2,502,France,Female,42,8,159660.80,3,1,0,113931.57
3,699,France,Female,39,1,0.00,2,0,0,93826.63
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10
...,...,...,...,...,...,...,...,...,...,...
9995,771,France,Male,39,5,0.00,2,1,0,96270.64
9996,516,France,Male,35,10,57369.61,1,1,1,101699.77
9997,709,France,Female,36,7,0.00,1,0,1,42085.58
9998,772,Germany,Male,42,3,75075.31,2,1,0,92888.52


In [316]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()

X['Gender'] = le.fit_transform(X['Gender'])

In [317]:
X

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,619,France,0,42,2,0.00,1,1,1,101348.88
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58
2,502,France,0,42,8,159660.80,3,1,0,113931.57
3,699,France,0,39,1,0.00,2,0,0,93826.63
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10
...,...,...,...,...,...,...,...,...,...,...
9995,771,France,1,39,5,0.00,2,1,0,96270.64
9996,516,France,1,35,10,57369.61,1,1,1,101699.77
9997,709,France,0,36,7,0.00,1,0,1,42085.58
9998,772,Germany,1,42,3,75075.31,2,1,0,92888.52


In [318]:
X = ct.fit_transform(X)

In [319]:
X

array([[1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00,
        1.0000000e+00, 1.0134888e+05],
       [0.0000000e+00, 0.0000000e+00, 1.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 1.1254258e+05],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00,
        0.0000000e+00, 1.1393157e+05],
       ...,
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 0.0000000e+00,
        1.0000000e+00, 4.2085580e+04],
       [0.0000000e+00, 1.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00,
        0.0000000e+00, 9.2888520e+04],
       [1.0000000e+00, 0.0000000e+00, 0.0000000e+00, ..., 1.0000000e+00,
        0.0000000e+00, 3.8190780e+04]])

In [320]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [321]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()


In [322]:
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [323]:
X_train.shape

(8000, 12)

## Building ANN

In [324]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense

In [325]:
model = keras.Sequential([
    layers.Dense(64, activation="relu", input_shape=(X_train.shape[1],)),
    layers.Dense(32, activation="relu"),
    layers.Dense(1, activation="sigmoid")
])


/usr/local/lib/python3.12/dist-packages/keras/src/layers/core/dense.py:93: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [326]:
model.compile(
    optimizer=keras.optimizers.Adam(
        learning_rate=0.001,
        beta_1=0.9,
        beta_2=0.999,
        epsilon=1e-8
    ),
    loss="binary_crossentropy",
    metrics=["accuracy"]
)

In [327]:
from tensorflow.keras.callbacks import EarlyStopping

early_stop = EarlyStopping(
    monitor='val_accuracy',
    patience=10,
    restore_best_weights=True
)
model.fit(
    X_train,
    y_train,
    batch_size=32,
    epochs=100,
    validation_split=0.2,
    callbacks=[early_stop]
)


Epoch 1/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.6862 - loss: 0.5653 - val_accuracy: 0.8194 - val_loss: 0.4292
Epoch 2/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8313 - loss: 0.3989 - val_accuracy: 0.8294 - val_loss: 0.3920
Epoch 3/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8544 - loss: 0.3631 - val_accuracy: 0.8419 - val_loss: 0.3708
Epoch 4/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8565 - loss: 0.3469 - val_accuracy: 0.8462 - val_loss: 0.3651
Epoch 5/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.8578 - loss: 0.3405 - val_accuracy: 0.8462 - val_loss: 0.3624
Epoch 6/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8604 - loss: 0.3366 - val_accuracy: 0.8450 - val_loss: 0.3606
Epoch 7/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 1s 3ms/step - accuracy: 0.8625 - loss: 0.3339 - val_accuracy: 0.8456 - val_loss: 0.3596
Epoch 8/100
200/200 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8633 - loss: 0.3313 - val_accu

In [328]:
model.history.history

{'accuracy': [0.7771875262260437,
  0.8335937261581421,
  0.8553125262260437,
  0.8582812547683716,
  0.8600000143051147,
  0.8610937595367432,
  0.8634374737739563,
  0.8643749952316284,
  0.8642187714576721,
  0.8648437261581421,
  0.8654687404632568,
  0.8673437237739563,
  0.8693749904632568,
  0.8704687356948853,
  0.870312511920929,
  0.8706250190734863,
  0.8712499737739563,
  0.8728125095367432,
  0.8743749856948853,
  0.8754687309265137,
  0.8767187595367432,
  0.8774999976158142,
  0.8776562213897705,
  0.8787500262260437,
  0.8793749809265137,
  0.8795312643051147,
  0.8793749809265137,
  0.8806250095367432,
  0.8812500238418579,
  0.8815624713897705,
  0.8812500238418579],
 'loss': [0.4813109338283539,
  0.3918078541755676,
  0.35893768072128296,
  0.34481528401374817,
  0.3386211097240448,
  0.3344840705394745,
  0.3315280079841614,
  0.3286619484424591,
  0.3262039124965668,
  0.32408368587493896,
  0.3220815658569336,
  0.32018133997917175,
  0.3184065818786621,
  0.3165

In [329]:
model.summary()

Model: "sequential_10"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ dense_24 (Dense)                │ (None, 64)             │           832 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_25 (Dense)                │ (None, 32)             │         2,080 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_26 (Dense)                │ (None, 1)              │            33 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 8,837 (34.52 KB)

 Trainable params: 2,945 (11.50 KB)

 Non-trainable params: 0 (0.00 B)

 Optimizer params: 5,892 (23.02 KB)

In [330]:
model.evaluate(X_test,y_test)

63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 2ms/step - accuracy: 0.8578 - loss: 0.3436


[0.34845170378685, 0.859000027179718]

In [331]:
# Raw input
custom = [[600,'France','Male',40,3,60000,2,1,1,50000]]

df_custom = pd.DataFrame(custom, columns=[
    'CreditScore', 'Geography', 'Gender', 'Age', 'Tenure', 'Balance',
    'NumOfProducts', 'HasCrCard', 'IsActiveMember', 'EstimatedSalary'
])

df_custom['Gender'] = le.transform(df_custom['Gender'])

df_custom = ct.transform(df_custom)

df_custom = sc.transform(df_custom)


In [332]:
ans = model.predict(df_custom)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step


In [333]:
ans

array([[0.02701475]], dtype=float32)

In [334]:
print("YES" if ans >= 0.5 else "NO")

NO


In [335]:
from sklearn.metrics import confusion_matrix, accuracy_score


In [336]:
y_pred = model.predict(X_test)
y_pred = (y_pred > 0.5)
y_true = y_test.values


63/63 ━━━━━━━━━━━━━━━━━━━━ 0s 1ms/step


In [337]:
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

cm = confusion_matrix(y_test, y_pred)
accuracy = accuracy_score(y_test, y_pred)

print("Confusion Matrix:")
print(cm)

print("Accuracy:", accuracy)

print("Classification Report:")
print(classification_report(y_test, y_pred))


Confusion Matrix:
[[1521   74]
 [ 208  197]]
Accuracy: 0.859
Classification Report:
              precision    recall  f1-score   support

           0       0.88      0.95      0.92      1595
           1       0.73      0.49      0.58       405

    accuracy                           0.86      2000
   macro avg       0.80      0.72      0.75      2000
weighted avg       0.85      0.86      0.85      2000



Precision, recall, and F1-score are metrics used to evaluate classification models, especially when the data is imbalanced.

Precision shows how often the model’s positive predictions are actually correct, while recall shows how many of the real positive cases the model is able to find. F1-score combines both precision and recall into a single value, giving a balanced measure of performance.

These metrics are more reliable than accuracy alone because accuracy can look high even when a model performs poorly on important classes. Precision, recall, and F1-score give deeper insight into the types of mistakes the model makes.

Other useful metrics include specificity, ROC-AUC, log loss, and MCC, which help evaluate different aspects of model performance.

## PyTorch

In [338]:
import random
import torch
seed = 42
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed_all(seed)

torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False


In [339]:
df = pd.read_csv("Churn_Modelling.csv")

X = df.drop(columns=["RowNumber", "CustomerId", "Surname", "Exited"])
y = df["Exited"]


X = pd.get_dummies(X, drop_first=True)


X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42
)


scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)


X_train = torch.tensor(X_train, dtype=torch.float32)
X_test  = torch.tensor(X_test, dtype=torch.float32)
y_train = torch.tensor(y_train.values, dtype=torch.float32).view(-1, 1)
y_test  = torch.tensor(y_test.values, dtype=torch.float32).view(-1, 1)


In [340]:
class ANN(nn.Module):
    def __init__(self, input_dim):
        super().__init__()
        self.net = nn.Sequential(
            nn.Linear(input_dim, 64),
            nn.ReLU(),
            nn.Linear(64, 32),
            nn.ReLU(),
            nn.Linear(32, 1),
            nn.Sigmoid()
        )

    def forward(self, x):
        return self.net(x)


In [341]:
model = ANN(X_train.shape[1])


In [342]:
criterion = nn.BCELoss()


In [343]:
optimizer = optim.Adam(
    model.parameters(),
    lr=0.001,
    betas=(0.9, 0.999),
    eps=1e-8   # match keras epsilon
)


In [344]:
epochs = 50
batch_size = 32

for epoch in range(epochs):
    model.train()

    permutation = torch.randperm(X_train.size(0))
    epoch_loss = 0

    for i in range(0, X_train.size(0), batch_size):
        indices = permutation[i:i+batch_size]
        batch_x = X_train[indices]
        batch_y = y_train[indices]

        optimizer.zero_grad()
        outputs = model(batch_x)
        loss = criterion(outputs, batch_y)
        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()

    if (epoch + 1) % 10 == 0:
        print(f"Epoch [{epoch+1}/{epochs}], Loss: {epoch_loss:.4f}")


Epoch [10/50], Loss: 82.8339
Epoch [20/50], Loss: 79.4290
Epoch [30/50], Loss: 76.8536
Epoch [40/50], Loss: 74.4159
Epoch [50/50], Loss: 72.5209


In [345]:
model.eval()

with torch.no_grad():
    y_pred_prob = model(X_test)
    y_pred = (y_pred_prob > 0.5).float()

accuracy = accuracy_score(
    y_test.numpy(),
    y_pred.numpy()
)

accuracy


0.856